In [1]:
import pandas as pd
import numpy as np
import os

from ratchet.Nephila import RiskStore

In [2]:
azure_server_name = r'nepsqlserverprod5.database.windows.net,1433'
azure_db_name = r'nepsqlserverprod5_DW'
azure_username = r'svcRatchetDW'
azure_password = r'1FF652D0-DE4C-407A-84A8-3726084A6266'

In [12]:
dcache_dir = r'E:\ELTCache'

In [3]:
xl_path = r'../data/static/testing/bert-reference-rollup-request.xlsx'
sheet_name = 'Portfolio Definition'

In [5]:
deal_data_df = pd.read_excel(xl_path, sheet_name, skiprows=15).set_index('Deal Reference')
deal_data_df['Deal Id'] = deal_data_df['Deal Id'].astype(str)

               Deal Source Deal Id Deal Type Inception Date Expiry Date  \
Deal Reference                                                            
2890-14-A              DMS   13484         S     2014-03-14  2018-04-06   
2890-14-B              DMS   13487         S     2014-03-14  2018-04-06   
5642-14-B              DMS   13900         S     2014-05-22  2018-05-25   
5642-14-C              DMS   13901         S     2014-05-22  2018-05-25   
5642-14-D              DMS   13902         S     2014-05-22  2019-05-28   
3321-14-B              DMS   14328         S     2014-05-30  2018-06-06   
6019-14-A              DMS   15040         S     2014-07-14  2019-07-13   
3007-14-A              DMS   15381         S     2014-09-16  2019-01-08   
7562-14                DMS   15735         S     2014-11-18  2019-11-25   
5828-14-A2             DMS   15926         S     2014-12-19  2019-01-16   
5828-14-B2             DMS   15927         S     2014-12-19  2020-01-16   
1923-17-J              DM

In [10]:
rs_linking_df = RiskStore.Sproc.get_linking_from_deal_list(
    azure_server_name, azure_db_name, azure_username, azure_password, 'DMS', 
    deal_data_df['Deal Id'].values, 210, as_of_date='2017-11-15 14:22:00')

In [44]:
if not RiskStore.ELTCache.check_cache_avail(dcache_dir):
    RiskStore.ELTCache.create_cache(dcache_dir)

In [13]:
RiskStore.ELTCache.load_missing_elts(
    azure_server_name, azure_db_name, azure_username, azure_password, rs_linking_df['EventSetId'].values, [51], dcache_dir)

In [18]:
deal_ref_deal_id_df = deal_data_df['Deal Id'].copy().reset_index()
deal_ref_deal_id_df.columns = ['DealRef', 'SourceDealId']

In [20]:
eventset_dealref_metarisk_df =\
    pd.merge(deal_ref_deal_id_df, rs_linking_df, left_on='SourceDealId', right_on='SourceDealId')

In [23]:
eventset_dealref_metarisk_df['MetaRiskId'] = 51

In [24]:
deal_elts_df = RiskStore.ELTCache.collect_cache_elts(eventset_dealref_metarisk_df, dcache_dir)

In [29]:
sells_idx = deal_data_df[deal_data_df['Deal Type'] == 'S'].index

limit_s = deal_data_df['Limit']
limit_s.loc[sells_idx] = -limit_s.loc[sells_idx]

rol_s = deal_data_df['ROL']

e:\anaconda2\envs\robocop\lib\site-packages\pandas\core\indexing.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [33]:
allocs_df = deal_data_df[[
    u'CCF Cronus',
    u'ECF Emerald',
    u'FCF Ferrous',
    u'HCF Hudson',
    u'ICF Iron',
    u'JCF Juniper',
    u'KCT Kumo Catastrophe Trust',
    u'LFL Lotus Master Fund Ltd',
    u'NCF Nephila',
    u'NSHF Nephila Syndicate Holdings',
    u'OCF Onyx',
    u'PCF Palmetto',
    u'RBK Rubik',
    u'RCF Rembrandt',
    u'TFL Triton',
    u'TYC Tyche',
    u'XCF Xanth',
    u'YCF Yderst'
]]

In [40]:
net_agg_51_ylt_df = RiskStore.ELTCache.calc_alloced_ylt(deal_elts_df, limit_s, rol_s, allocs_df, 'NET', 'AGG')
net_agg_51_ylt_df.to_csv('net_agg_51_ylt_df.csv')

In [41]:
gross_agg_51_ylt_df = RiskStore.ELTCache.calc_alloced_ylt(deal_elts_df, limit_s, rol_s, allocs_df, 'GROSS', 'AGG')
gross_agg_51_ylt_df.to_csv('gross_agg_51_ylt_df.csv')

In [42]:
net_occ_51_ylt_df = RiskStore.ELTCache.calc_alloced_ylt(deal_elts_df, limit_s, rol_s, allocs_df, 'NET', 'OCC')
net_occ_51_ylt_df.to_csv('net_occ_51_ylt_df.csv')

In [43]:
gross_occ_51_ylt_df = RiskStore.ELTCache.calc_alloced_ylt(deal_elts_df, limit_s, rol_s, allocs_df, 'GROSS', 'OCC')
gross_occ_51_ylt_df.to_csv('gross_occ_51_ylt_df.csv')